In [15]:
import os
import json
import pprint
import re
import argparse
from statistics import mean
from collections import Counter, defaultdict

import numpy as np
import matplotlib.pyplot as plt


In [16]:


# Tools
sca_tools = ['Grype.txt', 'Snyk.txt', 'Trivy.txt']

# RICs
rics = ['ONOS', 'OSC']

repoWithError = []

# Packages to exclude in the RIC repos
test_package = re.compile(r'test/')
benchmark_package = re.compile(r'benchmark')
examples_package = re.compile(r'examples/')
testapplication_package = re.compile(r'testapplication/')



In [115]:

# First we normalize the results from each tool
def format_sca_tool_data(repository, tool):
    if tool == "Grype.txt":
        return formatGrype(repository)
    elif tool == "Snyk.txt":
        return formatSnyk(repository)
    elif tool == "Trivy.txt":
        return formatTrivy(repository)

# This gets all the vulnerabilities in a normalized way.
# (In a list which includes all the vulnerabilities, which are not contained in a test package)
def formatGrype(repository):
    GrypeRepo = json.loads(repository)
    vulnArray = []
    for vuln in GrypeRepo["matches"]:
        path = vuln.get("artifact").get("locations")[0].get("path")
        if test_package.search(path) is not None:
            continue
        elif benchmark_package.search(path) is not None:
            continue
        elif examples_package.search(path) is not None:
            continue
        elif testapplication_package.search(path) is not None:
            continue
        else:
            vulnArray.append(vuln)
    return vulnArray

def formatSnyk(repository):
    content = json.loads(repository)
    vulnArray = []
    if "error" not in content:
        for target in content:
            if not isinstance(target, str):
                vulnList = target.get('vulnerabilities')
                path = target.get('displayTargetFile')
                if test_package.search(path) is not None:
                    print("Snyk: Skipping:" + path)
                    continue
                elif benchmark_package.search(path) is not None:
                    print("Snyk: Skipping:" + path)
                    continue
                elif examples_package.search(path) is not None:
                    print("Snyk: Skipping:" + path)
                    continue
                elif testapplication_package.search(path) is not None:
                    print("Snyk: Skipping:" + path)
                    continue
                else:
                    for vuln in vulnList:
                        vuln.pop('semver')
                        vulnArray.append(vuln)
            else:
                if target == 'vulnerabilities':
                    vulnList = content.get('vulnerabilities')
                    path = content.get('displayTargetFile')
                    print("Snyk path: {}".format(path))
                    if test_package.search(path) is not None:
                        print("Snyk: Skipping:" + path)
                        continue
                    elif benchmark_package.search(path) is not None:
                        print("Snyk: Skipping:" + path)
                        continue
                    elif examples_package.search(path) is not None:
                        print("Snyk: Skipping:" + path)
                        continue
                    elif testapplication_package.search(path) is not None:
                        print("Snyk: Skipping:" + path)
                        continue
                    else:
                        for vuln in vulnList:
                            vuln.pop('semver')
                            vulnArray.append(vuln)
                            print("1")
    else:
        global repoWithError
        repoWithError.append(os.path.basename(content['path']))
    return vulnArray

def formatTrivy(repository):
    index = repository.find("{")
    repo = repository[index:]
    TrivyRepo = json.loads(repo)
    results = TrivyRepo.get("Results")
    vulnArray = []
    if results is not None:
        for target in results:
            path = target.get("Target")
            if test_package.search(path) is not None:
                print("Trivy: Skipping:" + path)
                continue
            elif benchmark_package.search(path) is not None:
                print("Trivy: Skipping:" + path)
                continue
            elif examples_package.search(path) is not None:
                print("Trivy: Skipping:" + path)
                continue
            else:
                vulnTarget = target.get("Vulnerabilities", [])
                if not vulnTarget:
                    continue
                for vuln in vulnTarget:
                    vuln["Path"] = path
                vulnArray.extend(vulnTarget)
    return vulnArray

def save_vulnerabilities_by_directory(vulnerabilities_by_directory, base_dir="./ONOS"):
    for directory, vulnerabilities in vulnerabilities_by_directory.items():
        clean_directory = re.sub(r'[^a-zA-Z0-9_\-]', '', directory)
        dir_path = os.path.join(base_dir, clean_directory)
        os.makedirs(dir_path, exist_ok=True)
        
        filename = "Grype.json"
        filepath = os.path.join(dir_path, filename)
        
        with open(filepath, 'w') as json_file:
            json.dump(vulnerabilities, json_file, indent=4)

def dump_scan_results(rics, sca_tools):
    scan_results = dict.fromkeys(rics)
    onos_repos = []
    osc_repos = []
    for ric in rics:
        ric_dir = "./" + ric
        if not os.path.exists(ric_dir):
            os.makedirs(ric_dir)
        for repository in sorted(os.listdir(ric_dir)):
            if ric == "ONOS":
                onos_repos.append(repository)
            elif ric == "OSC":
                osc_repos.append(repository)
    for ric in rics:
        if ric == "ONOS":
            scan_results[ric] = dict.fromkeys(onos_repos)
        elif ric == "OSC":
            scan_results[ric] = dict.fromkeys(osc_repos)
        for repository in sorted(os.listdir("./" + ric)):
            scan_results[ric][repository] = dict.fromkeys(sca_tools)
            path_to_repository = os.path.join("./" + ric, repository)
            for sca_tool_file in sorted(os.listdir(path_to_repository)):
                sca_tool_file_path = os.path.join(path_to_repository, sca_tool_file)
                with open(sca_tool_file_path) as file:
                    vuln = file.read()
                #formatted_vuln = format_sca_tool_data(vuln, sca_tool_file)
                scan_results[ric][repository][sca_tool_file] = vuln
    with open('sca_results.json', 'w') as file:
        json.dump(scan_results, file)
    print("Finished writing: " + 'sca_results.json')
    pprint.pprint(scan_results)
    return scan_results

In [96]:
with open('results-26.json', 'r') as file:
        data = file.read()


In [97]:
print("Loading data...")

formatted_data = format_sca_tool_data(data, "Grype.txt")
print(formatted_data)

Loading data...
[{'vulnerability': {'id': 'GHSA-mvff-h3cj-wj9c', 'dataSource': 'https://github.com/advisories/GHSA-mvff-h3cj-wj9c', 'namespace': 'github:language:go', 'severity': 'High', 'urls': ['https://github.com/advisories/GHSA-mvff-h3cj-wj9c'], 'description': 'Unprivileged pod using `hostPath` can side-step active LSM when it is SELinux', 'cvss': [{'version': '3.1', 'vector': 'CVSS:3.1/AV:N/AC:H/PR:H/UI:N/S:C/C:H/I:H/A:H', 'metrics': {'baseScore': 8, 'exploitabilityScore': 1.3, 'impactScore': 6}, 'vendorMetadata': {'base_severity': 'High', 'status': 'N/A'}}], 'fix': {'versions': ['1.5.9'], 'state': 'fixed'}, 'advisories': []}, 'relatedVulnerabilities': [{'id': 'CVE-2021-43816', 'dataSource': 'https://nvd.nist.gov/vuln/detail/CVE-2021-43816', 'namespace': 'nvd:cpe', 'severity': 'Critical', 'urls': ['https://github.com/containerd/containerd/commit/a731039238c62be081eb8c31525b988415745eea', 'https://github.com/containerd/containerd/issues/6194', 'https://github.com/containerd/contain

In [116]:
formatted_data = format_sca_tool_data(data, "Grype.txt")
def get_vulnerabilities_by_directory(formatted_data):
        vulnerabilities_by_directory = defaultdict(list)
        
        for vuln in formatted_data:
                path = vuln.get("artifact").get("locations")[0].get("path")
                directory = os.path.dirname(path)
                vulnerabilities_by_directory[directory].append(vuln)

        return vulnerabilities_by_directory

def save_vulnerabilities_by_directory(vulnerabilities_by_directory, base_dir="./ONOS"):
    for directory, vulnerabilities in vulnerabilities_by_directory.items():
        
        clean_directory = re.sub(r'[^a-zA-Z0-9_\-]', '', directory)
        dir_path = os.path.join(base_dir, clean_directory)
        os.makedirs(dir_path, exist_ok=True)
        
        filename = "Grype.txt"
        filepath = os.path.join(dir_path, filename)
        
        with open(filepath, 'w') as json_file:
            json.dump(vulnerabilities, json_file, separators=(',', ':'))
        
        


vulnerabilities_by_directory = get_vulnerabilities_by_directory(formatted_data)
#pprint.pprint(vulnerabilities_by_directory)
save_vulnerabilities_by_directory(vulnerabilities_by_directory)


In [117]:
save_vulnerabilities_by_directory(vulnerabilities_by_directory)
dump_scan_results(['ONOS', 'OSC'], ['Grype.txt'])

Finished writing: sca_results.json
{'ONOS': {'onos-a1t': {'Grype.txt': '[{"vulnerability":{"id":"GHSA-mvff-h3cj-wj9c","dataSource":"https://github.com/advisories/GHSA-mvff-h3cj-wj9c","namespace":"github:language:go","severity":"High","urls":["https://github.com/advisories/GHSA-mvff-h3cj-wj9c"],"description":"Unprivileged '
                                    'pod using `hostPath` can side-step active '
                                    'LSM when it is '
                                    'SELinux","cvss":[{"version":"3.1","vector":"CVSS:3.1/AV:N/AC:H/PR:H/UI:N/S:C/C:H/I:H/A:H","metrics":{"baseScore":8,"exploitabilityScore":1.3,"impactScore":6},"vendorMetadata":{"base_severity":"High","status":"N/A"}}],"fix":{"versions":["1.5.9"],"state":"fixed"},"advisories":[]},"relatedVulnerabilities":[{"id":"CVE-2021-43816","dataSource":"https://nvd.nist.gov/vuln/detail/CVE-2021-43816","namespace":"nvd:cpe","severity":"Critical","urls":["https://github.com/containerd/containerd/commit/a731039238c

{'ONOS': {'onos-a1t': {'Grype.txt': '[{"vulnerability":{"id":"GHSA-mvff-h3cj-wj9c","dataSource":"https://github.com/advisories/GHSA-mvff-h3cj-wj9c","namespace":"github:language:go","severity":"High","urls":["https://github.com/advisories/GHSA-mvff-h3cj-wj9c"],"description":"Unprivileged pod using `hostPath` can side-step active LSM when it is SELinux","cvss":[{"version":"3.1","vector":"CVSS:3.1/AV:N/AC:H/PR:H/UI:N/S:C/C:H/I:H/A:H","metrics":{"baseScore":8,"exploitabilityScore":1.3,"impactScore":6},"vendorMetadata":{"base_severity":"High","status":"N/A"}}],"fix":{"versions":["1.5.9"],"state":"fixed"},"advisories":[]},"relatedVulnerabilities":[{"id":"CVE-2021-43816","dataSource":"https://nvd.nist.gov/vuln/detail/CVE-2021-43816","namespace":"nvd:cpe","severity":"Critical","urls":["https://github.com/containerd/containerd/commit/a731039238c62be081eb8c31525b988415745eea","https://github.com/containerd/containerd/issues/6194","https://github.com/containerd/containerd/security/advisories/GHSA

In [111]:
def get_cves_cvss_dependencies(sca_tool, sca_tool_data):
    cves_cvss_dependencies = []
    cves = []
    cvss = []
    packages = []
    if sca_tool == "Grype.txt":
        # print("Grype")
        for vulnerability in sca_tool_data:
            if vulnerability.get("vulnerability").get("id") not in cves:
                cves.append(vulnerability.get("vulnerability").get("id"))
                cvss.append(vulnerability.get("vulnerability").get("cvss")[0].get("metrics").get("baseScore")) 
                vulnerability_match_details = vulnerability.get("matchDetails")
                for match_detail in vulnerability_match_details:
                    # print("Printing vulnerability details for:" + str(match_detail))
                    if "package" in match_detail["searchedBy"].keys():
                        packages.append(match_detail["searchedBy"]["package"]["name"])
                    elif "Package" in match_detail["searchedBy"].keys():
                        packages.append(match_detail["searchedBy"]["Package"]["name"])
            else:
                continue
        cves_cvss_dependencies = [cves, cvss, packages]
        return cves_cvss_dependencies
    elif sca_tool == "Snyk.txt":
        print("Snyk")
        for vulnerability in sca_tool_data:
            if len(vulnerability.get("identifiers").get("CVE")) == 0:
                continue
                # cves.append(None)
                # cvss.append(None)
                # packages.append(None)
            else:
                if vulnerability.get("identifiers").get("CVE")[0] not in cves:
                    cves.append(vulnerability.get("identifiers").get("CVE")[0])
                    cvss.append(vulnerability.get("cvssScore"))
                    packages.append(vulnerability.get("moduleName"))
        print("number of unique vulnerabilities: " + str(len(cves)))
        cves_cvss_dependencies = [cves, cvss, packages]
        return cves_cvss_dependencies
    elif sca_tool == "Trivy.txt":
        # print("Trivy")
        for vulnerability in sca_tool_data:
            if vulnerability.get("VulnerabilityID") not in cves:
                if vulnerability.get("CVSS") is not None:
                    cves.append(vulnerability.get("VulnerabilityID"))
                    packages.append(vulnerability.get("PkgName"))
                    nvd = vulnerability.get("CVSS").get("nvd")
                    ghsa = vulnerability.get("CVSS").get("ghsa")
                    if nvd is not None:
                        cvss.append(nvd.get("V3Score"))
                        continue
                    elif ghsa is not None:
                        cvss.append(ghsa.get("V3Score"))
                        continue
                else:
                    print("Printing a vuln with None score")
                    pprint.pprint(vulnerability)
                    continue
        cves_cvss_dependencies = [cves, cvss, packages]
        return cves_cvss_dependencies
    else:
        print("Unknown tool")
    return cves_cvss_dependencies

In [127]:
with open('sca_results.json', 'r') as file:
    sca_results = json.load(file)
exm = sca_results["ONOS"]["onos-a1t"]["Grype.txt"]   
print(exm) 
cves_cvss_dependencies = get_cves_cvss_dependencies("Grype.txt", exm)
#get_cves_cvss_dependencies("Grype.txt", sca_results["ONOS"]["onos-a1t"]["Grype.txt"])

[{"vulnerability":{"id":"GHSA-mvff-h3cj-wj9c","dataSource":"https://github.com/advisories/GHSA-mvff-h3cj-wj9c","namespace":"github:language:go","severity":"High","urls":["https://github.com/advisories/GHSA-mvff-h3cj-wj9c"],"description":"Unprivileged pod using `hostPath` can side-step active LSM when it is SELinux","cvss":[{"version":"3.1","vector":"CVSS:3.1/AV:N/AC:H/PR:H/UI:N/S:C/C:H/I:H/A:H","metrics":{"baseScore":8,"exploitabilityScore":1.3,"impactScore":6},"vendorMetadata":{"base_severity":"High","status":"N/A"}}],"fix":{"versions":["1.5.9"],"state":"fixed"},"advisories":[]},"relatedVulnerabilities":[{"id":"CVE-2021-43816","dataSource":"https://nvd.nist.gov/vuln/detail/CVE-2021-43816","namespace":"nvd:cpe","severity":"Critical","urls":["https://github.com/containerd/containerd/commit/a731039238c62be081eb8c31525b988415745eea","https://github.com/containerd/containerd/issues/6194","https://github.com/containerd/containerd/security/advisories/GHSA-mvff-h3cj-wj9c","https://github.com/

AttributeError: 'str' object has no attribute 'get'

In [128]:
with open('sca_results.json', 'r') as file:
    sca_results = json.load(file)

# Specify the RIC, repository, and tool you want to analyze
ric = "ONOS"  # Replace with the actual RIC you are interested in
repository = "onos-a1t"  # Replace with your repository name
tool = "Grype.txt"  # Replace with the actual tool name you are analyzing

# Extract the relevant data from the JSON
sca_tool_data_str = sca_results[ric][repository][tool]

# Parse the extracted JSON string
sca_tool_data = json.loads(sca_tool_data_str)

# Call the function with the parsed data
result = get_cves_cvss_dependencies(tool, sca_tool_data)
print(result)

IndexError: list index out of range